In [1]:
%%capture
!pip install torch
!pip install transformers
!pip install datasets
!pip install rouge-score

In [2]:
from google.colab import drive
import shutil

# Монтируем Google Drive
drive.mount('/content/gdrive')

# Путь для сохранения модели на Google Drive
drive_path = '/content/gdrive/MyDrive/summarization_model_bart'

Mounted at /content/gdrive


In [3]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
from rouge_score import rouge_scorer
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

# Загрузка датасета
dataset = load_dataset('IlyaGusev/gazeta', revision='v2.0')

# Ограничение выборки
small_dataset = dataset['train'].shuffle(seed=42).select([i for i in range(50000)])

# Использование BART-модели для суммаризации
model_name = 'facebook/bart-base'
#model_name = 'facebook/bart-small'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['summary']

    inputs = [text.strip() for text in inputs]
    targets = [summary.strip() for summary in targets]

    # Tokenization
    inputs = tokenizer(inputs, return_tensors='pt', truncation='only_first', padding='max_length')
    targets = tokenizer(targets, return_tensors='pt', truncation='only_first', padding='max_length')

    return {**inputs, 'labels': targets['input_ids']}

# Обработка данных
train_data = small_dataset.map(preprocess_function, batched=True)

# Перенос модели на GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Обучение модели
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# Определение оптимизатора и функции потерь
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Обработка данных
train_data = small_dataset.map(preprocess_function, batched=True)

# Перенос модели на GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Обучение модели
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in tqdm(torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True), desc=f'Epoch {epoch + 1}', leave=False):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_data)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}')

# Сохранение модели
model.save_pretrained('summarization_model_bart')
tokenizer.save_pretrained('summarization_model_bart')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnin

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Epoch 1/3, Average Loss: 0.3217


Epoch 2/3, Average Loss: 0.2092


Epoch 3/3, Average Loss: 0.1888


('summarization_model_bart/tokenizer_config.json',
 'summarization_model_bart/special_tokens_map.json',
 'summarization_model_bart/vocab.json',
 'summarization_model_bart/merges.txt',
 'summarization_model_bart/added_tokens.json')

In [4]:
# Создаем директорию
shutil.rmtree(drive_path, ignore_errors=True)
shutil.copytree('summarization_model_bart', drive_path)

print(f"Модель сохранена в {drive_path}")

Модель сохранена в /content/gdrive/MyDrive/summarization_model_bart


In [5]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
from rouge_score import rouge_scorer

# Load the saved model and tokenizer
model_name = 'summarization_model_bart'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Move the model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load the dataset for evaluation
eval_dataset = load_dataset('IlyaGusev/gazeta', revision='v2.0', split='validation')

# Limit the evaluation dataset
small_eval_dataset = eval_dataset.shuffle(seed=42).select([i for i in range(1000)])

# Preprocess the evaluation dataset
def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['summary']

    inputs = [text.strip() for text in inputs]
    targets = [summary.strip() for summary in targets]

    inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=1024, return_tensors='pt')
    targets = tokenizer(targets, padding='max_length', truncation=True, max_length=1024, return_tensors='pt')

    return {**inputs, 'labels': targets['input_ids']}

eval_data = small_eval_dataset.map(preprocess_function, batched=True)
eval_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Generate summaries using the model
generated_summaries = []
for batch in torch.utils.data.DataLoader(eval_data, batch_size=4):
    batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to the same device as the model
    summaries = model.generate(**batch)
    generated_summaries.extend(tokenizer.batch_decode(summaries, skip_special_tokens=True))

# Join generated summaries into a single string
generated_summaries_str = ' '.join(generated_summaries)

# Join reference summaries into a single string
references_str = ' '.join(small_eval_dataset['summary'])

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = scorer.score(generated_summaries_str, references_str)

# Print the ROUGE scores
print("ROUGE-1: {:.4f}".format(rouge_scores['rouge1'].fmeasure))
print("ROUGE-2: {:.4f}".format(rouge_scores['rouge2'].fmeasure))
print("ROUGE-L: {:.4f}".format(rouge_scores['rougeL'].fmeasure))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ROUGE-1: 0.1315
ROUGE-2: 0.0255
ROUGE-L: 0.0980


In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.nn.utils.rnn import pad_sequence

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка сохраненной модели и токенизатора
loaded_tokenizer = BartTokenizer.from_pretrained('/content/gdrive/MyDrive/summarization_model_bart')
loaded_model = BartForConditionalGeneration.from_pretrained('/content/gdrive/MyDrive/summarization_model_bart')

# Пример текста для суммаризации
input_text = """
В последние десятилетия технологии претерпели огромные изменения, повлияв на все аспекты нашей жизни.
С развитием интернета и мобильных устройств информация стала более доступной, а общение с людьми по всему миру — проще.
Технологические инновации также сказались на образовании, предоставляя новые возможности для учебы и развития.
В современном обществе образование играет ключевую роль.
Оно не только передает знания, но и способствует личному развитию.
Студенты имеют доступ к разнообразным образовательным ресурсам в Интернете, что позволяет им глубже изучать предметы, которые их интересуют.
Виртуальные классы и дистанционные образовательные программы расширяют границы обучения, делая его более гибким и доступным.
Современные технологии также влияют на методы преподавания.
Интерактивные доски, мультимедийные материалы и онлайн-тестирование делают процесс обучения более интересным и эффективным.
Учители могут создавать динамичные уроки, а студенты — взаимодействовать с материалом в более глубоком понимании.
"""
# Токенизация и подготовка данных для модели
input_ids = loaded_tokenizer(input_text, return_tensors='pt', truncation=True, padding=True)['input_ids']
input_ids = input_ids.to(device)  # Ensure input tensors are on the same device as the model

# Генерация суммаризации
summary_ids = loaded_model.generate(input_ids.to(device), max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
summary = loaded_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Вывод результатов
print("Исходный текст:")
print(input_text)
print("\nСгенерированная суммаризация:")
print(summary)
print("Оригинальное саммари")
print("""
Современные технологии, включая интернет и мобильные устройства, переформатируют образование, предоставляя учащимся доступ к образовательным ресурсам.
Обучение становится более гибким благодаря виртуальным классам и дистанционным программам
""")

Исходный текст:

В последние десятилетия технологии претерпели огромные изменения, повлияв на все аспекты нашей жизни. 
С развитием интернета и мобильных устройств информация стала более доступной, а общение с людьми по всему миру — проще. 
Технологические инновации также сказались на образовании, предоставляя новые возможности для учебы и развития.
В современном обществе образование играет ключевую роль. 
Оно не только передает знания, но и способствует личному развитию. 
Студенты имеют доступ к разнообразным образовательным ресурсам в Интернете, что позволяет им глубже изучать предметы, которые их интересуют. 
Виртуальные классы и дистанционные образовательные программы расширяют границы обучения, делая его более гибким и доступным.
Современные технологии также влияют на методы преподавания. 
Интерактивные доски, мультимедийные материалы и онлайн-тестирование делают процесс обучения более интересным и эффективным. 
Учители могут создавать динамичные уроки, а студенты — взаимодействов